
| Leader     | Customer1  | Customer2  | Customer3  
|------------|---------|----------|------------
| Terry  | 10       | 15        | 9          
| Carle    | 9       | 18        | 5          
| McClymonds      | 6       | 14        | 3          



In [2]:
from itertools import product

import gurobipy as gp
from gurobipy import GRB



In [21]:
agents = 'Terry Carle McClymonds'.split()
num_customers = 3


pairs, times = gp.multidict({
    p:t for p,t in zip([*product(agents, range(1, num_customers + 1))], [10, 15, 9, 9, 18, 5, 6, 14, 3]) 
})


<gurobi.tuplelist (9 tuples, 2 values each):
 ( Terry      , 1 )
 ( Terry      , 2 )
 ( Terry      , 3 )
 ( Carle      , 1 )
 ( Carle      , 2 )
 ( Carle      , 3 )
 ( McClymonds , 1 )
 ( McClymonds , 2 )
 ( McClymonds , 3 )
>


In [11]:

m = gp.Model('Assignment_Model')

In [22]:
assignment = m.addVars(pairs, vtype=GRB.CONTINUOUS, name='assignment')

In [24]:
m.setObjective(gp.quicksum(times[i,j] * assignment[i,j] for i,j in pairs), GRB.MINIMIZE)

In [25]:
agents = m.addConstrs(assignment.sum(i, '*') <= 1 for i in agents  )

In [26]:
tasks = m.addConstrs(assignment.sum('*', j) == 1 for j in range(1, num_customers + 1))

In [27]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 6 rows, 27 columns and 18 nonzeros
Model fingerprint: 0x0b202db3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 18 columns
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3000000e+01   2.000000e+00   0.000000e+00      0s
       2    2.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.600000000e+01


In [28]:
print(f'Objective Value: {m.ObjVal} days')

Objective Value: 26.0 days


In [30]:
for v in m.getVars():
    if v.x > 0.00001:
        print(f'{v.varName}: {v.x}')

assignment[Terry,2]: 1.0
assignment[Carle,3]: 1.0
assignment[McClymonds,1]: 1.0


In [32]:
print(m.NumVars)

27


In [33]:
print(m.NumConstrs)

6
